In [1]:
!pip install pandas

  Using cached pandas-1.2.4-cp39-cp39-manylinux1_x86_64.whl (9.7 MB)
  Using cached numpy-1.20.3-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.4 MB)
  Using cached pytz-2021.1-py2.py3-none-any.whl (510 kB)


In [34]:
!pip install matplotlib

     |████████████████████████████████| 10.3 MB 3.7 MB/s eta 0:00:01
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
  Using cached kiwisolver-1.3.1-cp39-cp39-manylinux1_x86_64.whl (1.2 MB)
     |████████████████████████████████| 3.0 MB 3.3 MB/s eta 0:00:01


!pip freeze

# Librerías

In [281]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

DEBUG = True

# Carga de datos

In [282]:
DATADIR = 'data'
FILENAME = 'challenge_orders.csv'
PATH = os.path.join(DATADIR, FILENAME)

In [283]:
df = pd.read_csv(PATH, index_col=0, parse_dates=['CREATED_AT'])

## Preprocesamiento

In [284]:
df.rename(columns={'ORDER_ID':'order', 
                   'PRODUCT_ID':'product', 
                   'PAYMENT_METHOD':'pay', 
                   'TOTAL_VALUE':'total', 
                   'CREATED_AT':'created', 
                   'STORE_TO_USER_DISTANCE':'dist', 
                   'OBSF_USER_ID':'user', 
                   'OBSF_STORE_TYPE':'store'}, inplace=True)
df.total = df.total.astype(int)
df.dist = df.dist.astype(int)

# Necesidades de negocio

## Ranking por tipo de tienda que más dólares generó
Se agrupó por ```store```, sumando sus valores y luego ordenandolos de mayor a menor.

In [285]:
def get_revenue_per_store(df:pd.DataFrame)->pd.DataFrame:
    revenue = df.groupby(by='store').sum().total.sort_values(ascending=False).to_frame() \
    .rename(columns={'total':'revenue'})
    return revenue

In [286]:
revenue = get_revenue_per_store(df)

In [287]:
revenue.head()

,revenue
store,
1,201929817
13,21517956
9,18329730
4,5827298
5,4301348


## Proporción por medio de pago
Primero se filtro por medio de pago, *i.e.* ```pay``` y luego se agrupó por ```store``` calculando la suma total. Este procedimiento se debe realizar dos veces, una por método de pago, pero con la función ```get_revenue_by_pay_method``` es posible obtener este cálculo extendiendolo a $n$ métodos de pago.

### Método 1

In [ ]:
def get_revenue_by_pay_method(df:pd.DataFrame, pay_method:str)->pd.DataFrame:
    # Filter and group-by
    gb = df[df.pay==pay_method].groupby(by='store').sum().total
    # Transform to DataFrame
    revenue_per_pay_method = gb.to_frame().rename(columns={'total':f'revenue_{pay_method}'})
    # Match total and revenue per pay method
    return revenue_per_pay_method

In [ ]:
# Get revenue per payment method
revenue_cc = get_revenue_by_pay_method(df,'cc')
revenue_cash = get_revenue_by_pay_method(df,'cash')

In [ ]:
# Concat dfs
my_df = pd.concat([gen_usd, revenue_cc, revenue_cash], axis=1).fillna(0).astype(int)

In [ ]:
# Revenue per df
my_df['pct_cc'] = my_df.revenue_cc/my_df.revenue
my_df['pct_cash'] = my_df.revenue_cash/my_df.revenue

In [ ]:
my_df

### Método 2

In [288]:
def get_pct_by_pay_method(df:pd.DataFrame, pay_method:str)->pd.DataFrame:
    # Filter and group-by
    revenue_per_store = get_revenue_per_store(df)
    revenue_per_store_filtered = df[df.pay==pay_method].groupby(by='store').sum().total.to_frame() \
    .rename(columns={'total':f'revenue_{pay_method}'})
    # Match total and revenue per pay method
    temp_df = pd.concat([revenue_per_store, revenue_per_store_filtered], axis=1).fillna(0)
    # Show temporal df
    if DEBUG:
        display(temp_df.head(10))
    pct_by_pay_method = (temp_df[f'revenue_{pay_method}']/temp_df['revenue']).to_frame() \
    .rename(columns={0:f'pct_{pay_method}'})
    return pct_by_pay_method

In [289]:
pct_by_pay_cc = get_pct_by_pay_method(df,'cc')
pct_by_pay_cash = get_pct_by_pay_method(df,'cash')

,revenue,revenue_cc
store,,
1,201929817,186318196.0
2,3871475,3341701.0
3,4000246,2787398.0
4,5827298,4986456.0
5,4301348,3339639.0
6,146305,103575.0
7,3801,3801.0
8,3092649,2620032.0
9,18329730,16188011.0


,revenue,revenue_cash
store,,
1,201929817,15611621.0
2,3871475,529774.0
3,4000246,1212848.0
4,5827298,840842.0
5,4301348,961709.0
6,146305,42730.0
7,3801,0.0
8,3092649,472617.0
9,18329730,2141719.0


In [290]:
# Create ranking
ranking_df = pd.concat([revenue, pct_by_pay_cc, pct_by_pay_cash], axis=1).fillna(0)

In [291]:
ranking_df.sort_values(by='revenue', ascending=False).head(10)

,revenue,pct_cc,pct_cash
store,,,
1,201929817,0.922688,0.077312
13,21517956,0.862548,0.137452
9,18329730,0.883156,0.116844
4,5827298,0.855706,0.144294
5,4301348,0.776417,0.223583
12,4242174,0.837108,0.162892
3,4000246,0.696807,0.303193
2,3871475,0.863160,0.136840
8,3092649,0.847181,0.152819


## AOV
El ```aov``` (valor medio de la orden) se calcula de la siguiente manera:
$$\texttt{AOV} = \frac{\texttt{revenue}}{\texttt{orders}}$$

In [276]:
def get_orders_per_store(df:pd.DataFrame)->pd.DataFrame:
    orders = df.groupby(by='store').count().order.to_frame().rename(columns={'order':'orders'})
    return orders

In [319]:
def get_aov_per_store(df:pd.DataFrame)->pd.DataFrame:
    orders = get_orders_per_store(df)
    revenue = get_revenue_per_store(df)
    temp_df = pd.concat([orders, revenue], axis=1)
    if DEBUG:
        #display(temp_df)
        pass
    aov = (temp_df.revenue/temp_df.orders).to_frame().rename(columns={0:'aov'})
    return aov

In [321]:
aov = get_aov_per_store(df)

In [324]:
ranking_df = ranking_df.join(aov)

In [327]:
ranking_df.sort_values(by='revenue', ascending=False).head(10)

,revenue,pct_cc,pct_cash,aov
store,,,,
1,201929817,0.922688,0.077312,31147.588616
13,21517956,0.862548,0.137452,17200.604317
9,18329730,0.883156,0.116844,13577.577778
4,5827298,0.855706,0.144294,8951.302611
5,4301348,0.776417,0.223583,5828.384824
12,4242174,0.837108,0.162892,5466.719072
3,4000246,0.696807,0.303193,8511.161702
2,3871475,0.863160,0.136840,10784.052925
8,3092649,0.847181,0.152819,3944.705357
